In [11]:
from sentence_transformers import util
from transformers import AutoTokenizer, AutoModel
import torch
import json
import time
from kiwipiepy import Kiwi
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
def replacecomma(m):        # 매개변수로 매치 객체를 받음
    n = m.group().replace(",","")    # 매칭된 문자열
    return str(n)

print("로딩 중(~5초)")
kiwi = Kiwi()
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-nli')
model = AutoModel.from_pretrained('jhgan/ko-sbert-nli')
#model = SentenceTransformer("Huffon/sentence-klue-roberta-base")

HUGGINGFACE_MODEL_PATH = "bespin-global/klue-bert-base-mrc"
tokenizer2 = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH )
model2 = AutoModelForQuestionAnswering.from_pretrained(HUGGINGFACE_MODEL_PATH )

#입출력 파일
infile = 'q1.json'
outfile = 'a1.json'
outfile4 = 'a4.json'
def bespin(context, question):
    # Encoding
    encodings = tokenizer2(context, question, 
                          max_length=512, 
                          truncation=True,
                          padding="max_length", 
                          return_token_type_ids=False
                          )
    encodings = {key: torch.tensor([val]) for key, val in encodings.items()}             
    input_ids = encodings["input_ids"]
    attention_mask = encodings["attention_mask"]

    # Predict
    pred = model2(input_ids, attention_mask=attention_mask)

    start_logits, end_logits = pred.start_logits, pred.end_logits
    token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
    pred_ids = input_ids[0][token_start_index: token_end_index + 1]

    # Decoding
    prediction = tokenizer2.decode(pred_ids)
    if(prediction.find('[SEP]')>0):
        prediction = prediction[0:prediction.find('[SEP]')]
    return prediction

def encode(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    #encoded_input2 = tokenizer(query, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        #model_output2 = model(**encoded_input2)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    #query_embedding = mean_pooling(model_output2, encoded_input2['attention_mask'])
    return sentence_embeddings;



with open(infile, encoding='UTF-8') as f:
    #json_data = json.load(f)
    json_data =json.loads(f.read())
    #lines = f.readlines()

    
'''
cont=""
for line in lines:
    cont = cont+line;

json_data = json.loads(cont)
'''
json_out = []
json_out4 = []
cnt = 0
for d in json_data:
    cnt = cnt+1
    start = time.time()
    problem_no = d['problem_no']
    task_no = d['task_no']
    doc_file = d['doc_file']
    question = d['question']
    paras = d['paras']
    sent = d['sent']
    para = d['para']
    pat = d['pat']
    #question 틀린점 있어서 마지막 한단어 제거함
    tok = question.split(" ")
    question = " ".join(tok[:-1])
    
    query = question
    sentences = paras
    ####sentences.append(question)
    '''
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    encoded_input2 = tokenizer(query, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        model_output2 = model(**encoded_input2)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    query_embedding = mean_pooling(model_output2, encoded_input2['attention_mask'])
    '''
    sentence_embeddings = encode(sentences);
    query_embedding = encode(query);
    #sentence_embeddings = model_output[:-1]
    #query_embedding = model_output[-1]

    top_k = min(3, len(sentences))
    cos_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    a = {}
    a['team_id']=""
    a['hash']=""
    a['problem_no']=problem_no
    a['task_no']=task_no
    
    a['doc_file']=doc_file
    a['question'] = question
    a['answer_right'] = 0
    a['para'] = para
    answer = []
    for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
        rank = str(i+1);
        paragraph = sentences[idx]
        if(para==paragraph):
            a['answer_right']=i+1
        
        ans={}
        ans['rank']=rank
        ans['paragraph']=paragraph
        answer.append(ans)
    a['answer']=answer
    a['sent'] = sent
    delta = str(time.time()-start)
    a['time']=delta
    
    #검색 응답 출력
    json_out.append(a);
    
    #찾은 문단에서 QA 시작
    pred_para = answer[0]['paragraph'];
    sentences = kiwi.split_into_sents(pred_para)
    newsents = []
    for s in sentences:
        newsents.append(s.text);
    sentences = newsents
    
    if(sentences==[]):
        continue;
    if(len(sentences[0])==0):
        continue;
    
    sentence_embeddings = encode(sentences);
    top_k = min(1, len(sentences))
    cos_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    top1sent = ""
    for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
        rank = str(i+1);
        top1sent = sentences[idx]
    
    pred_qa = bespin(top1sent,question);
    
    #deepcopy보다 빠른 json 복사
    #data_copy = json.loads(json.dumps(data))
    a = json.loads(json.dumps(a))
    
    a['answer'] = pred_qa
    a['task_no'] = "4"
    evidence = []
    e1 = {}
    e1['doc']=doc_file
    e2 = {}
    e2['자료 유형(paragraph)']=pred_para
    e3 = {}
    e3['자료 유형(table/figure)']=""
    evidence.append(e1)
    evidence.append(e2)
    evidence.append(e3)
    a['evidence']=evidence
    delta = str(time.time()-start)
    a['time']=delta
    json_out4.append(a);
    
    
    
    #if(cnt%500==0):
    if(cnt%2==0):
        out = open(outfile,'w')
        out.write(json.dumps(json_out, ensure_ascii=False, indent=4))
        out.close();
        
        out4 = open(outfile4,'w')
        out4.write(json.dumps(json_out4, ensure_ascii=False, indent=4))
        out4.close();
        
        print("\n")
        print(str(cnt)+"/"+str(len(json_data))+" "+str(delta)[:4]+"초");
        print(f"Question: {query}")
        print("Pred_qa: "+pred_qa);
        #print(f"<입력 문장과 유사한 {top_k} 개의 문장>")

        for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
            print(f"[{i+1}] {sentences[idx][0:min(500,len(sentences[idx]))]} {'(유사도: {:.4f})'.format(score)}")

        



로딩 중(~5초)


2/27983 15.3초
Question: 실험식이를 진행한 12주 동안 mouse body weight 변화 (n=6-9
Pred_qa: mice / group
[1] 실험식이를 진행한 12주 동안 mouse body weight 변화 (n=6-9 mice/group), *: HFD군과 비교하여 유의적인 차이 (*:p<0.05, **:p<0.01, ***:p<0.001, ****:p<0.0001) (유사도: 0.8470)


4/27983 3.65초
Question: (Arg1 상승이 유지됨)l
Pred_qa: 
[1] l GNP-CD163를 처리해주었을 경우, 다른 성상의 macrophages 에 비해 TAM2 (M2 macrophages)에서 GNP uptake 가 증가한 것을 IF Staining으로 확인하였고, TEM에서도 확인함. (유사도: 0.4909)


6/27983 14.2초
Question: 또한 예주권역은 영해면의 중심부에 위치하여 영덕군청에서 북쪽으로 약 15km거리에 있으며 동쪽으로는
Pred_qa: 동해바다
[1] 영덕군 영해면 성내리, 괴시리, 연평리 일원(법정리 3개, 행정리 10개, 자연마을 12개) 면적825ha(농경지:340ha, 임야:316ha, 기타:169ha)으로 예주권역은 영해면의 중심부에 위치하여 영덕군청에서 북쪽으로 약 15km거리에 있으며 동쪽으로는 동해바다, 서쪽으로는 창수면, 남쪽으로는 축산면, 북쪽으로는 병곡면과 접해 있습니다. (유사도: 0.8002)


8/27983 5.54초
Question: 대상자의 자기관리 교육 요구도 항목별 중요도 순위는 1) 증상 및
Pred_qa: 징후
[1] 대상자의 자기관리 교육 요구도 항목별 중요도 순위는 1) 증상 및 징후, 2) 약물관련 정보, 3) 질병의 예후, 4) 위험요인, 5) 일반적 질병 정보, 6) 식이, 7) 신체활동, 8) 심리관련 정보 순이었다 (유사도: 0.8663)


10/27983 13.7초
Question: ‘Halla 

IndexError: list index out of range